In [ ]:
cd /kaggle/input/mancode1/MAN

In [ ]:
import torch
import cv2
import numpy as np
from PIL import Image
from torchvision import transforms
import os
from zipfile import ZipFile
from tqdm import tqdm
from models.vgg_c import vgg19_trans
import glob

class VideoCrowdCounter:
    def __init__(self, model_path, device='cuda'):
        self.device = torch.device(device if torch.cuda.is_available() else 'cpu')
        self.model = self.load_model(model_path)
        self.transform = transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ])

    def load_model(self, model_path):
        model = vgg19_trans()
        model.load_state_dict(torch.load(model_path, self.device, weights_only=True))
        model = model.to(self.device)
        model.eval()
        return model

    def process_frame(self, frame):
        # Convert frame to PIL Image
        img = Image.fromarray(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
        
        # Transform image
        img_tensor = self.transform(img).unsqueeze(0)
        img_tensor = img_tensor.to(self.device)
        
        # Get prediction
        with torch.no_grad():
            density_map, _ = self.model(img_tensor)
        
        # Process density map
        density_map = density_map.squeeze().cpu().numpy()
        count = np.sum(density_map)
        
        # Resize density map to match frame size
        density_map = cv2.resize(density_map, (frame.shape[1], frame.shape[0]))
        normalized_map = np.clip(density_map/np.max(density_map), 0, 1)
        
        # Create heatmap
        heatmap = cv2.applyColorMap((normalized_map * 255).astype(np.uint8), cv2.COLORMAP_JET)
        
        # Create overlay
        overlay = cv2.addWeighted(frame, 0.6, heatmap, 0.4, 0)
        
        # Add count text to original frame
        count_frame = frame.copy()
        cv2.putText(count_frame, f'Count: {int(count)}', (30, 30),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
        
        return count_frame, heatmap, overlay

    def process_video(self, video_path, output_dir, fps=30):
        # Get video name without extension
        video_name = os.path.splitext(os.path.basename(video_path))[0]
        
        # Create output directory if it doesn't exist
        os.makedirs(output_dir, exist_ok=True)
        
        try:
            # Open video
            cap = cv2.VideoCapture(video_path)
            if not cap.isOpened():
                print(f"Error: Could not open video {video_path}")
                return None
                
            total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
            width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
            height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
            
            # Create output filenames
            count_path = os.path.join(output_dir, f'{video_name}_count.mp4')
            density_path = os.path.join(output_dir, f'{video_name}_density.mp4')
            overlay_path = os.path.join(output_dir, f'{video_name}_overlay.mp4')
            
            # Create video writers
            fourcc = cv2.VideoWriter_fourcc(*'mp4v')
            count_writer = cv2.VideoWriter(count_path, fourcc, fps, (width, height))
            density_writer = cv2.VideoWriter(density_path, fourcc, fps, (width, height))
            overlay_writer = cv2.VideoWriter(overlay_path, fourcc, fps, (width, height))
            
            # Process each frame
            pbar = tqdm(total=total_frames, desc=f'Processing {video_name}')
            while cap.isOpened():
                ret, frame = cap.read()
                if not ret:
                    break
                    
                # Process frame
                count_frame, density_map, overlay = self.process_frame(frame)
                
                # Write frames
                count_writer.write(count_frame)
                density_writer.write(density_map)
                overlay_writer.write(overlay)
                
                pbar.update(1)
            
            # Release everything
            cap.release()
            count_writer.release()
            density_writer.release()
            overlay_writer.release()
            pbar.close()
            
            return [count_path, density_path, overlay_path]
            
        except Exception as e:
            print(f"Error processing video {video_path}: {str(e)}")
            return None

def process_videos_in_directory(input_dir, model_path, output_dir):
    # Initialize processor
    processor = VideoCrowdCounter(model_path)
    
    # Get all video files
    video_files = glob.glob(os.path.join(input_dir, "*.mp4")) + \
                 glob.glob(os.path.join(input_dir, "*.avi")) + \
                 glob.glob(os.path.join(input_dir, "*.mov"))
    
    if not video_files:
        print(f"No video files found in {input_dir}")
        return
    
    all_output_files = []
    
    # Process each video
    for video_path in video_files:
        output_files = processor.process_video(video_path, output_dir)
        if output_files:
            all_output_files.extend(output_files)
    
    # Create zip file if we have processed files
    if all_output_files:
        zip_path = os.path.join(output_dir, 'crowd_vid4.zip')
        with ZipFile(zip_path, 'w') as zipf:
            for file in all_output_files:
                if os.path.exists(file):
                    zipf.write(file, os.path.basename(file))
        print(f"Results saved to {zip_path}")
    else:
        print("No videos were successfully processed")

def main():
    # Configure paths
    input_dir = '/kaggle/input/stadium-vids'
    model_path = '/kaggle/input/man-model/model.pth'
    output_dir = '/kaggle/working'
    
    process_videos_in_directory(input_dir, model_path, output_dir)

if __name__ == "__main__":
    main()